In [102]:
import nltk, re
from nltk.corpus import stopwords

### Проверка орфографии

1. Название должности/позиции (% грамотности в названии должности)

In [1]:
import re
import pandas as pd
import enchant
from enchant.checker import SpellChecker
from nltk.tokenize import ToktokTokenizer

# инициализируем словари
d_ru = enchant.DictWithPWL('ru')

#инициализируем токенайзер
toktok = ToktokTokenizer()

In [2]:
train = pd.read_csv('train.csv', sep=';')
test = pd.read_csv('test.csv', sep=';')

C:\Users\BikmetovaMR\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.shape, test.shape

((306270, 25), (131259, 24))

In [20]:
def spell_check_position(line):
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', line.lower())
    
    chkr_ru = SpellChecker(d_ru)
    chkr_ru.set_text(line)
    mistakes_ru = set([err.word for err in chkr_ru])
    
    
    tokenized = toktok.tokenize(line)
    num_mistakes=0
    for i in tokenized:
        if i in mistakes_ru:
            num_mistakes+=1
            
    num_words = len(line.split(' '))
    
    grammar_rate = num_mistakes / num_words
            
    return grammar_rate

In [21]:
%%time
train['grammar_per_cent'] = train['position'].apply(spell_check_position)

Wall time: 3min 30s


In [22]:
%%time
test['grammar_per_cent'] = test['position'].apply(spell_check_position)

Wall time: 1min 28s


In [23]:
train_v3 = pd.read_csv('train_v3.csv')
test_v3 = pd.read_csv('test_v3.csv')

In [27]:
train.rename(columns={"grammar_per_cent": "grammar_mistakes_per_cent"}, inplace=True)
test.rename(columns={"grammar_per_cent": "grammar_mistakes_per_cent"}, inplace=True)

In [28]:
train['grammar_mistakes_per_cent'].value_counts()

0.000000    279154
0.500000      6258
1.000000      5106
0.333333      3094
0.250000      2472
             ...  
0.875000         1
0.161290         1
0.217391         1
0.004651         1
0.228571         1
Name: grammar_mistakes_per_cent, Length: 158, dtype: int64

In [31]:
pd.merge(train[['id', 'grammar_mistakes_per_cent']], train_v3).to_csv('train_v4.csv',index=False)

In [32]:
pd.merge(test[['id', 'grammar_mistakes_per_cent']], test_v3).to_csv('test_v4.csv',index=False)

### Фича - Образование - вуз или ссуз

In [34]:
for col in train.columns:
    if 'edu' in col:
        print(col)

education_type
schedule


In [37]:
train_v3['education_type_le']

0         Незаконченное высшее
1                            3
2                            3
3                            3
4                          NaN
                  ...         
306265                       2
306266                       3
306267                       3
306268                       3
306269                       2
Name: education_type_le, Length: 306270, dtype: object

In [53]:
empl = pd.read_csv('employements.csv',sep=';')
empl.shape

(343328, 7)

In [73]:
len(empl['id'].unique())

343328

In [54]:
empl.columns

Index(['id', 'position', 'employer', 'achievements', 'responsibilities',
       'start_date', 'finish_date'],
      dtype='object')

In [60]:
empl.head()

,id,position,employer,achievements,responsibilities,start_date,finish_date
0,0,NaN,"ПАО ""МегаФон""",NaN,<p>Консультирование клиентов по телефону. Прод...,2019-06-01,2020-04-01
1,1,Копирайтер/рерайтер,WorkHardOnline,NaN,<p><strong>✑</strong><strong>Фриланс</strong><...,2019-10-01,NaN
2,2,стажер,Комитет по управлению муниципальной собственно...,NaN,<p>Прохождение производственной практики (30.0...,2014-06-01,2014-07-01
3,3,Консультант,ПАО Почта Банк,NaN,"<p>Работа с клиентами, оформление договоров</p>",2016-04-01,NaN
4,4,Старший специалист по привлечению клиентов,"ОАО Банк ""Открытие""",<p>Расширила базу клиентов</p>,"<p>Обслуживание физических лиц, активное привл...",2011-07-01,2013-01-01


In [69]:
# 3086 пропуски
empl['achievements'].isna().sum()

3086

In [113]:
empl['ach_clean'] = empl['achievements'].apply(clean_text)

In [178]:
def no_achievements(line):
    if 'нет' in line or 'не им' in line or 'нет' in line or 'никак' in line or 'отсутств' in line\
    or 'увол по собствен желан' in line or 'не работа' in line or 'не был' in line or 'без достижен' in line:
        return 'нет'
    else:
        return line

In [179]:
empl['ach_clean_no'] = empl['ach_clean'].apply(no_achievements)

In [186]:
def three_cat_achieve(line):
    if line =='':
        return 'empty'
    elif line =='нет':
        return 'no'
    else:
        return 'yes'

In [189]:
empl['achievement'] = empl['ach_clean_no'].apply(three_cat_achieve)

In [192]:
empl_achiev = pd.get_dummies(empl[['id','achievement']])

In [204]:
empl_achiev

,id,achievement_empty,achievement_no,achievement_yes
0,0,1,0,0
1,1,1,0,0
2,2,1,0,0
3,3,1,0,0
4,4,0,0,1
...,...,...,...,...
343323,437521,1,0,0
343324,437522,1,0,0
343325,437524,1,0,0
343326,437525,0,0,1


In [221]:
train_v6 = pd.read_csv('train_v6', sep=';')
test_v6 = pd.read_csv('test_v6', sep=';')


train_v6.shape, test_v6.shape

((306270, 104), (131259, 104))

In [222]:
train_achieve = pd.merge(train_v6, empl_achiev, how='left')

In [223]:
test_achieve = pd.merge(test_v6, empl_achiev, how='left')

In [231]:
train_achieve.to_csv('train_v7.csv',index=False)

In [230]:
test_achieve.to_csv('test_v7.csv',index=False)

In [226]:
test_achieve['achievement_empty'].fillna(1,inplace=True)
test_achieve['achievement_no'].fillna(0,inplace=True)
test_achieve['achievement_yes'].fillna(0,inplace=True)

In [227]:
train_achieve['achievement_empty'].fillna(1,inplace=True)
train_achieve['achievement_no'].fillna(0,inplace=True)
train_achieve['achievement_yes'].fillna(0,inplace=True)

In [85]:
empl['responsibilities']

pandas.core.series.Series

In [109]:
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords

stopwords = stopwords.words('russian')

toktok = ToktokTokenizer()
stemmer = RussianStemmer()

# text = re.sub('[^А-Яа-я]+', ' ', text)
# all_words = nltk.tokenize.word_tokenize(text.lower())
# all_words_no_stop = [w.lower() for w in all_words if w not in stopwords]
# freq_dict = nltk.FreqDist(w.lower() for w in all_words if w not in stopwords)

def clean_text(line):
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', str(line).lower())
    
    line = re.sub('[^А-Яа-я]+', ' ', line)
#     all_words = nltk.tokenize.word_tokenize(line.lower())
    tokenized = toktok.tokenize(line)
    stemmed_str = ''
    for i in tokenized:
        stemmed = stemmer.stem(i)
        stemmed_str += str(stemmed + ' ')
    
    return stemmed_str

In [110]:
empl['resp_lower'] = empl['responsibilities'].apply(clean_text)

In [89]:
empl['resp_lower'].value_counts()

<p>ведение кадрового делопроизводства</p>               46
<p>работать</p>                                         46
<p>продажи&nbsp,</p>                                    46
<p>выполнение должностных обязанностей</p>              46
<p>бухгалтерский учет</p>                               46
<p>дворник</p>                                          45
<p>оператор</p>                                         45
<p>уход за больными</p>                                 45
<p>управление автомобилем</p>                           44
<p>охрана обьекта</p>                                   44
<p>расследование уголовных дел</p>                      44
<p>&nbsp,</p><p>&nbsp,</p>                              44
<p>ремонт</p>                                           44
<p>пошив одежды</p>                                     43
работа с документами                                    43
<p>продажа банковских продуктов</p>                     42
<p>уборка номеров</p>                                   

In [100]:
pd.merge(empl[['id','resp_lower']], train[['id', 'position']])

,id,resp_lower,position
0,0,<p>консультирование клиентов по телефону. прод...,Специалист
1,1,<p><strong>✑</strong><strong>фриланс</strong><...,Лаборант
2,3,"<p>работа с клиентами, оформление договоров</p>","Специалист, администратор, бухгалтер, экономист"
3,4,"<p>обслуживание физических лиц, активное привл...",Специалист
4,5,<p>проведение обучающих и оздоровительных заня...,Инструктор по плаванию
...,...,...,...
240417,437519,"<p>стрижки, прически, окрашивание.</p>",Главный специалист государственной или муницип...
240418,437521,"<p>скупка электроники, продажа, конструкция кл...",Продавец консультант.
240419,437522,"<p>пожаротушение, осушествление противопожарно...",менеджер
240420,437524,"<p>разработка документации, в соотвествии с тр...",инженер
